### import the dependencies 

In [45]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Perprocessing

In [46]:
# loading the data frame from csv file to pandas dataframe
df = pd.read_csv(r"E:\project on programmming\14_Movie Recommendation System\data\movies.csv")
# printing the first 5 rows of the dataframe
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [47]:
# numbers of rows and columns in dataframe 
df.shape

(4803, 24)

In [48]:
# checking if there is missing data 
df.isnull().sum()


index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [50]:
# checking for duplicated data 
df.duplicated().sum()

np.int64(0)

In [51]:
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [52]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [53]:
# selecting the relevant features from dataframe
selected_features = ['genres' ,'keywords','overview', 'tagline', 'cast' , 'director' ]

In [54]:
# replacing the null values with null string
for feature in selected_features :
    df[feature] = df[feature].fillna('')
    print(df[feature].dtypes)

object
object
object
object
object
object


In [55]:
combined_features = df[selected_features].apply(lambda x: ' '.join(x), axis=1)



In [56]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [57]:
combined_features = combined_features.str.replace(r'\\u[0-9A-Fa-f]{4}', '', regex=True)


In [58]:
combined_features = combined_features.str.lower()


In [59]:
import re
combined_features = combined_features.apply(lambda x: re.sub('[^a-zA-Z ]', ' ', x))


In [60]:
combined_features = combined_features.str.replace(r'\s+', ' ', regex=True)


In [61]:
# convert the text data to feature vectorizer

vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1,2),
    min_df=2
)


In [62]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 187133 stored elements and shape (4803, 5000)>
  Coords	Values
  (0, 19)	0.06632128025324069
  (0, 54)	0.07603438345113998
  (0, 1584)	0.0945506758202384
  (0, 3977)	0.08820052644367443
  (0, 1635)	0.08840404735907156
  (0, 1029)	0.16645092765362612
  (0, 815)	0.17203586625278425
  (0, 1766)	0.12321912605540856
  (0, 4205)	0.27063681299399367
  (0, 4812)	0.09592131288087954
  (0, 868)	0.19237746046863252
  (0, 4173)	0.14845441349226843
  (0, 703)	0.1388386324759044
  (0, 2836)	0.159084173581199
  (0, 3056)	0.16402928259580585
  (0, 4684)	0.17203586625278425
  (0, 3026)	0.12148991401605323
  (0, 4569)	0.16180158509050976
  (0, 1695)	0.15379500143353134
  (0, 3277)	0.17903028887039119
  (0, 3563)	0.1849113085721328
  (0, 122)	0.13644305895269102
  (0, 807)	0.18332051211335643
  (0, 1452)	0.16109695603277968
  (0, 4940)	0.08077018202867582
  :	:
  (4802, 526)	0.36639149706583996
  (4802, 2674)	0.12910682728029807
  (4802, 1532)

### Cosine Similarity 

In [63]:
# getting the similarity scores using cosine similarity
similarity  = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.04636987 0.03480259 ... 0.00628929 0.03138532 0.        ]
 [0.04636987 1.         0.01389444 ... 0.02530428 0.01421503 0.        ]
 [0.03480259 0.01389444 1.         ... 0.01337365 0.06470612 0.        ]
 ...
 [0.00628929 0.02530428 0.01337365 ... 1.         0.02796921 0.03602399]
 [0.03138532 0.01421503 0.06470612 ... 0.02796921 1.         0.01605643]
 [0.         0.         0.         ... 0.03602399 0.01605643 1.        ]]


In [64]:
print(similarity.shape)

(4803, 4803)


In [65]:
# geting the movies name from the user 
movie_name = input ('Enter your favorite movie name :')


In [66]:
# creating a list with all movies names given in the dataset
list_to_all_titles = df['title'].tolist()
list_to_all_titles 


['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [67]:
# find the close match for movie name
find_close_match = difflib.get_close_matches(movie_name , list_to_all_titles)
if len(find_close_match) == 0:
    print("Movie not found. Please check the spelling.")
else:
    close_match = find_close_match[0]
    print("Closest match:", close_match)

Closest match: Iron Man


In [68]:
# finding the index of movies with title
index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)


68


In [69]:
similarity_score = list (enumerate(similarity[index_of_the_movie]))
similarity_score

[(0, np.float64(0.05601531918297179)),
 (1, np.float64(0.03225704886080443)),
 (2, np.float64(0.00856155549266291)),
 (3, np.float64(0.0027981367648760858)),
 (4, np.float64(0.025517217639700954)),
 (5, np.float64(0.007459611455293941)),
 (6, np.float64(0.021555231262485212)),
 (7, np.float64(0.26747019997961824)),
 (8, np.float64(0.0057974689770722235)),
 (9, np.float64(0.039786088908411385)),
 (10, np.float64(0.0652526674191602)),
 (11, np.float64(0.008172263666383283)),
 (12, np.float64(0.009182885588612705)),
 (13, np.float64(0.007271039531706427)),
 (14, np.float64(0.06173061474296422)),
 (15, np.float64(0.01569272962795804)),
 (16, np.float64(0.2325182146728197)),
 (17, np.float64(0.012547472054146235)),
 (18, np.float64(0.015578339012479096)),
 (19, np.float64(0.044539419739065556)),
 (20, np.float64(0.056051664019259345)),
 (21, np.float64(0.023708619291200037)),
 (22, np.float64(0.005598958100342735)),
 (23, np.float64(0.011176030050480347)),
 (24, np.float64(0.008487346764921

In [70]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))


In [71]:
similarity_score


[(0, np.float64(0.05601531918297179)),
 (1, np.float64(0.03225704886080443)),
 (2, np.float64(0.00856155549266291)),
 (3, np.float64(0.0027981367648760858)),
 (4, np.float64(0.025517217639700954)),
 (5, np.float64(0.007459611455293941)),
 (6, np.float64(0.021555231262485212)),
 (7, np.float64(0.26747019997961824)),
 (8, np.float64(0.0057974689770722235)),
 (9, np.float64(0.039786088908411385)),
 (10, np.float64(0.0652526674191602)),
 (11, np.float64(0.008172263666383283)),
 (12, np.float64(0.009182885588612705)),
 (13, np.float64(0.007271039531706427)),
 (14, np.float64(0.06173061474296422)),
 (15, np.float64(0.01569272962795804)),
 (16, np.float64(0.2325182146728197)),
 (17, np.float64(0.012547472054146235)),
 (18, np.float64(0.015578339012479096)),
 (19, np.float64(0.044539419739065556)),
 (20, np.float64(0.056051664019259345)),
 (21, np.float64(0.023708619291200037)),
 (22, np.float64(0.005598958100342735)),
 (23, np.float64(0.011176030050480347)),
 (24, np.float64(0.008487346764921

In [72]:
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)


In [73]:
sorted_similar_movies

[(68, np.float64(1.0)),
 (79, np.float64(0.4075056960523269)),
 (31, np.float64(0.310694439241972)),
 (7, np.float64(0.26747019997961824)),
 (16, np.float64(0.2325182146728197)),
 (26, np.float64(0.22070057214788993)),
 (511, np.float64(0.18169809474961676)),
 (182, np.float64(0.1787024697379824)),
 (101, np.float64(0.1745806053735454)),
 (3623, np.float64(0.16599521429987613)),
 (94, np.float64(0.16313957986654506)),
 (203, np.float64(0.15480060086737055)),
 (64, np.float64(0.15200320615122648)),
 (174, np.float64(0.14695645802657364)),
 (3466, np.float64(0.14571700254596812)),
 (1740, np.float64(0.14491174354600758)),
 (232, np.float64(0.13467812326709855)),
 (205, np.float64(0.1309763501966455)),
 (4364, np.float64(0.12891503499950793)),
 (126, np.float64(0.127920252074088)),
 (3414, np.float64(0.12784416574122415)),
 (2235, np.float64(0.12478485573813049)),
 (85, np.float64(0.12426779439637917)),
 (3385, np.float64(0.12391553143063515)),
 (788, np.float64(0.1228515996694331)),
 (33

In [74]:
# sorting the movies bsed on their similarity score 
similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

# print the similarity moves based on index 
for i in range(1, 11):
    movie_index = sorted_similar_movies[i][0]
    print(df.iloc[movie_index]['title'])


Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
X-Men
Ant-Man
X-Men: First Class
Made
Guardians of the Galaxy


In [75]:
def recommend_movies(movie_name):
    
    find_close_match = difflib.get_close_matches(movie_name, df['title'].tolist())
    
    if not find_close_match:
        return "Movie not found"
    
    close_match = find_close_match[0]
    index_of_the_movie = df[df.title == close_match].index[0]
    
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    print("Movies suggested for you:\n")
    
    for i in range(1, min(11, len(sorted_similar_movies))):
        print(df.iloc[sorted_similar_movies[i][0]]['title'])


### Testing section

In [76]:
# normal test case
recommend_movies('batman')

Movies suggested for you:

Batman Returns
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Forever
The Dark Knight
Batman: The Dark Knight Returns, Part 2
Suicide Squad
Batman v Superman: Dawn of Justice
Beetlejuice


In [77]:
# incorrect  test case 
recommend_movies('Iorn Mn')


Movies suggested for you:

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
X-Men
Ant-Man
X-Men: First Class
Made
Guardians of the Galaxy


In [78]:
# movie is not found test case 
recommend_movies('abcdefg')


'Movie not found'

In [79]:
# little information test case
recommend_movies('Super Size Me')


Movies suggested for you:

Food, Inc.
Larry the Cable Guy: Health Inspector
Harold & Kumar Go to White Castle
Cloudy with a Chance of Meatballs
Cloudy with a Chance of Meatballs 2
The Harvest (La Cosecha)
Foodfight!
Fuel
Clerks II
The Greatest Movie Ever Sold


In [80]:
# iinternal test inportant 
print(feature_vectors.shape)
print(similarity.shape)


(4803, 5000)
(4803, 4803)


In [81]:
import pickle

pickle.dump(similarity, open(r'E:\project on programmming\14_Movie Recommendation System\models\similarity.pkl', 'wb'))
pickle.dump(vectorizer, open(r'E:\project on programmming\14_Movie Recommendation System\models\vectorizer.pkl', 'wb'))
